In [1]:
import pandas as pd
import numpy as np
import requests
import json
from sklearn.preprocessing import LabelEncoder, StandardScaler

**https://developer.here.com/documentation/traffic-api/dev_guide/topics/concepts/flow.html#flow**

# HERE Traffic flow API
**Traffic flow data that is similar to DataMall Speed Bands, but instead of a range (minimum and maximum speed), it provides the speed itself and also a jam factor that tells us how congested it is**

**It also covers more roads (1500+) depending on how we set the parameter**

**The "in=circle:1.279683,103.809628" parameter in the request url represents a point in the map, and r=20830 represents the radius**

In [2]:
response = requests.get("https://data.traffic.hereapi.com/v7/flow?in=circle:1.279683,103.809628;r=20830&locationReferencing=shape&apiKey=uAF0ydU9hJ_THCHkwvtSQR5FhTKEWjtX8XynT5rHoPk")

In [3]:
data = response.json()["results"]

In [4]:
data

[{'location': {'description': 'Changi South Ave 2',
   'length': 92.0,
   'shape': {'links': [{'points': [{'lat': 1.3317399937659502,
        'lng': 103.96608003415167},
       {'lat': 1.3319799676537514, 'lng': 103.96619997918606},
       {'lat': 1.3322200253605843, 'lng': 103.96634004078805},
       {'lat': 1.3324599992483854, 'lng': 103.9664899930358}],
      'length': 92.0}]}},
  'currentFlow': {'speed': 11.666667,
   'speedUncapped': 11.666667,
   'freeFlow': 10.277778,
   'jamFactor': 0.0,
   'confidence': 0.92,
   'traversability': 'open'}},
 {'location': {'description': 'ECP/East Coast Pkwy',
   'length': 44.0,
   'shape': {'links': [{'points': [{'lat': 1.3121199700981379,
        'lng': 103.95154003053904},
       {'lat': 1.3120300322771072, 'lng': 103.95136996172369},
       {'lat': 1.311940010637045, 'lng': 103.95119000226259}],
      'length': 44.0}]}},
  'currentFlow': {'speed': 17.777779,
   'speedUncapped': 17.777779,
   'freeFlow': 13.888889,
   'jamFactor': 0.0,
   'co

In [5]:
raw_df = pd.DataFrame(data)

In [6]:
raw_df = pd.concat([raw_df['location'].apply(pd.Series), raw_df['currentFlow'].apply(pd.Series)], axis=1)

**Extract the beginning and end points of the road**

In [7]:
raw_df['start_lat'] = raw_df['shape'][0]['links'][0]['points'][0]['lat']
raw_df['start_lon'] = raw_df['shape'][0]['links'][0]['points'][0]['lng']
raw_df['end_lat'] = raw_df['shape'][0]['links'][0]['points'][-1]['lat']
raw_df['end_lon'] = raw_df['shape'][0]['links'][0]['points'][-1]['lng']

In [8]:
raw_df.head()

,description,length,shape,speed,speedUncapped,freeFlow,jamFactor,confidence,traversability,subSegments,junctionTraversability,start_lat,start_lon,end_lat,end_lon
0,Changi South Ave 2,92.0,{'links': [{'points': [{'lat': 1.3317399937659...,11.666667,11.666667,10.277778,0.0,0.92,open,NaN,NaN,1.33174,103.96608,1.33246,103.96649
1,ECP/East Coast Pkwy,44.0,{'links': [{'points': [{'lat': 1.3121199700981...,17.777779,17.777779,13.888889,0.0,0.70,open,NaN,NaN,1.33174,103.96608,1.33246,103.96649
2,Benoi Rd,511.0,{'links': [{'points': [{'lat': 1.3087099604308...,8.888889,8.888889,12.500000,1.9,0.76,open,NaN,NaN,1.33174,103.96608,1.33246,103.96649
3,Jalan Kebun Limau,457.0,{'links': [{'points': [{'lat': 1.3168200384825...,9.166667,9.166667,13.055556,1.8,0.99,open,NaN,NaN,1.33174,103.96608,1.33246,103.96649
4,Jurong Town Hall Rd/Exit 13,1734.0,{'links': [{'points': [{'lat': 1.3186700083315...,21.388890,21.388890,20.833334,0.0,0.99,open,NaN,NaN,1.33174,103.96608,1.33246,103.96649


# Pre-Processing Data

### Handle missing values

**Drop the shape feature**

In [9]:
raw_df = raw_df.drop("shape", axis=1)

**Drop subSegments, junctionTraversability, jamTendency, as only a small number of records have these features**

In [10]:
columns_to_drop = ['subSegments', 'junctionTraversability', 'jamTendency']

In [11]:
columns_to_drop = [col for col in columns_to_drop if col in raw_df.columns]

In [12]:
raw_df = raw_df.drop(columns_to_drop, axis=1)

**Roads that are closed have no traffic features**

In [13]:
null_rows = raw_df['speed'].isnull()
null_data = raw_df[null_rows]

In [14]:
null_data.head()

,description,length,speed,speedUncapped,freeFlow,jamFactor,confidence,traversability,start_lat,start_lon,end_lat,end_lon
162,Paisr Ris Dr 3/New Loyang Link,386.0,NaN,NaN,11.111112,10.0,NaN,closed,1.33174,103.96608,1.33246,103.96649
514,Pasir Ris Dr 4/Pasir Ris St 12,405.0,NaN,NaN,11.944445,10.0,NaN,closed,1.33174,103.96608,1.33246,103.96649
1450,Jalan Lingkaran Dalam,174.0,NaN,NaN,9.166667,10.0,NaN,closed,1.33174,103.96608,1.33246,103.96649
1732,Loyang Way,632.0,NaN,NaN,14.166667,10.0,NaN,closed,1.33174,103.96608,1.33246,103.96649
2620,Marymount Rd,354.0,NaN,NaN,9.722222,10.0,NaN,closed,1.33174,103.96608,1.33246,103.96649


**Select rows where traversibility is open**

In [15]:
open_df = raw_df.loc[raw_df['traversability'] == "open"].copy()

In [16]:
open_df = open_df.drop("traversability", axis=1)

In [17]:
open_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5506 entries, 0 to 5515
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   description    5502 non-null   object 
 1   length         5506 non-null   float64
 2   speed          5506 non-null   float64
 3   speedUncapped  5506 non-null   float64
 4   freeFlow       5506 non-null   float64
 5   jamFactor      5506 non-null   float64
 6   confidence     5506 non-null   float64
 7   start_lat      5506 non-null   float64
 8   start_lon      5506 non-null   float64
 9   end_lat        5506 non-null   float64
 10  end_lon        5506 non-null   float64
dtypes: float64(10), object(1)
memory usage: 516.2+ KB


**There are still some records without a road name description**

In [18]:
open_df.dropna(inplace=True)

In [19]:
open_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5502 entries, 0 to 5515
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   description    5502 non-null   object 
 1   length         5502 non-null   float64
 2   speed          5502 non-null   float64
 3   speedUncapped  5502 non-null   float64
 4   freeFlow       5502 non-null   float64
 5   jamFactor      5502 non-null   float64
 6   confidence     5502 non-null   float64
 7   start_lat      5502 non-null   float64
 8   start_lon      5502 non-null   float64
 9   end_lat        5502 non-null   float64
 10  end_lon        5502 non-null   float64
dtypes: float64(10), object(1)
memory usage: 515.8+ KB


In [20]:
road_list = open_df["description"].unique().tolist()

In [21]:
road_list = sorted([str(item) for item in road_list])

In [22]:
len(road_list)

1538

In [23]:
for item in road_list:
    if type(item) == float:
        print("hey")

In [24]:
df = open_df.copy()

**Drop road metadata**

In [25]:
#df = df.drop("olr", axis=1)

**Confidence <br>
<br>
0.7 < confidence <= 1.0 indicates real time speeds <br>
0.5 < confidence <= 0.7 indicates historical speeds <br>
0.0 < confidence <= 0.5 indicates speed limit**

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5502 entries, 0 to 5515
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   description    5502 non-null   object 
 1   length         5502 non-null   float64
 2   speed          5502 non-null   float64
 3   speedUncapped  5502 non-null   float64
 4   freeFlow       5502 non-null   float64
 5   jamFactor      5502 non-null   float64
 6   confidence     5502 non-null   float64
 7   start_lat      5502 non-null   float64
 8   start_lon      5502 non-null   float64
 9   end_lat        5502 non-null   float64
 10  end_lon        5502 non-null   float64
dtypes: float64(10), object(1)
memory usage: 515.8+ KB


In [27]:
df.head()

,description,length,speed,speedUncapped,freeFlow,jamFactor,confidence,start_lat,start_lon,end_lat,end_lon
0,Changi South Ave 2,92.0,11.666667,11.666667,10.277778,0.0,0.92,1.33174,103.96608,1.33246,103.96649
1,ECP/East Coast Pkwy,44.0,17.777779,17.777779,13.888889,0.0,0.70,1.33174,103.96608,1.33246,103.96649
2,Benoi Rd,511.0,8.888889,8.888889,12.500000,1.9,0.76,1.33174,103.96608,1.33246,103.96649
3,Jalan Kebun Limau,457.0,9.166667,9.166667,13.055556,1.8,0.99,1.33174,103.96608,1.33246,103.96649
4,Jurong Town Hall Rd/Exit 13,1734.0,21.388890,21.388890,20.833334,0.0,0.99,1.33174,103.96608,1.33246,103.96649


In [28]:
#df["confidence"] = pd.cut(df["confidence"], bins=[0.0, 0.5, 0.7, 1.0], labels=["Speed Limit", "Historical Speeds", "Real-Time Speeds"])

In [29]:
#df = pd.get_dummies(df, columns=["confidence"])

In [30]:
#df.head()

In [31]:
#df.to_csv('./test.csv', index=False)

In [32]:
#label_encoder = LabelEncoder()
#df['traversability'] = label_encoder.fit_transform(df['traversability'])

In [33]:
df.head()

,description,length,speed,speedUncapped,freeFlow,jamFactor,confidence,start_lat,start_lon,end_lat,end_lon
0,Changi South Ave 2,92.0,11.666667,11.666667,10.277778,0.0,0.92,1.33174,103.96608,1.33246,103.96649
1,ECP/East Coast Pkwy,44.0,17.777779,17.777779,13.888889,0.0,0.70,1.33174,103.96608,1.33246,103.96649
2,Benoi Rd,511.0,8.888889,8.888889,12.500000,1.9,0.76,1.33174,103.96608,1.33246,103.96649
3,Jalan Kebun Limau,457.0,9.166667,9.166667,13.055556,1.8,0.99,1.33174,103.96608,1.33246,103.96649
4,Jurong Town Hall Rd/Exit 13,1734.0,21.388890,21.388890,20.833334,0.0,0.99,1.33174,103.96608,1.33246,103.96649


In [34]:
'''
yo = requests.get("https://data.traffic.hereapi.com/v7/flow?in=circle:1.279683,103.809628;r=20830&locationReferencing=olr&apiKey=uAF0ydU9hJ_THCHkwvtSQR5FhTKEWjtX8XynT5rHoPk")
ya = yo.json()["results"]
ye = pd.DataFrame(ya)
ye = pd.concat([ye['location'].apply(pd.Series)])
ye.head()
head1 = df["description"].unique().tolist()
head2 = ye["description"].unique().tolist()
len(head1)
len(head2)

if set(head1) == set(head2):
    print("The lists have the same contents.")
else:
    print("The lists have different contents.")
'''

'\nyo = requests.get("https://data.traffic.hereapi.com/v7/flow?in=circle:1.279683,103.809628;r=20830&locationReferencing=olr&apiKey=uAF0ydU9hJ_THCHkwvtSQR5FhTKEWjtX8XynT5rHoPk")\nya = yo.json()["results"]\nye = pd.DataFrame(ya)\nye = pd.concat([ye[\'location\'].apply(pd.Series)])\nye.head()\nhead1 = df["description"].unique().tolist()\nhead2 = ye["description"].unique().tolist()\nlen(head1)\nlen(head2)\n\nif set(head1) == set(head2):\n    print("The lists have the same contents.")\nelse:\n    print("The lists have different contents.")\n'

# Handling duplicate data

In [35]:
duplicates = df.duplicated(subset='description', keep=False)
duplicate_df = df[duplicates]

In [36]:
duplicate_df

,description,length,speed,speedUncapped,freeFlow,jamFactor,confidence,start_lat,start_lon,end_lat,end_lon
0,Changi South Ave 2,92.0,11.666667,11.666667,10.277778,0.0,0.92,1.33174,103.96608,1.33246,103.96649
2,Benoi Rd,511.0,8.888889,8.888889,12.500000,1.9,0.76,1.33174,103.96608,1.33246,103.96649
3,Jalan Kebun Limau,457.0,9.166667,9.166667,13.055556,1.8,0.99,1.33174,103.96608,1.33246,103.96649
4,Jurong Town Hall Rd/Exit 13,1734.0,21.388890,21.388890,20.833334,0.0,0.99,1.33174,103.96608,1.33246,103.96649
5,Syed Alwi Rd,35.0,5.277778,5.277778,7.777778,1.7,0.84,1.33174,103.96608,1.33246,103.96649
...,...,...,...,...,...,...,...,...,...,...,...
5510,TOH TUCK AVE NB AND TOH TUCK AVE SB,144.0,13.333334,13.333334,13.888889,0.3,0.99,1.33174,103.96608,1.33246,103.96649
5511,CTE/Central Expy,892.0,11.944445,11.944445,12.222222,0.3,0.93,1.33174,103.96608,1.33246,103.96649
5512,Tavistock Ave/Berwick Dr,31.0,6.388889,6.388889,6.388889,0.0,0.70,1.33174,103.96608,1.33246,103.96649
5514,Marina Grv,146.0,13.611112,13.611112,11.111112,0.0,0.99,1.33174,103.96608,1.33246,103.96649


In [37]:
unique_df = df[~df.duplicated(subset='description', keep=False)]

In [38]:
unique_df

,description,length,speed,speedUncapped,freeFlow,jamFactor,confidence,start_lat,start_lon,end_lat,end_lon
1,ECP/East Coast Pkwy,44.0,17.777779,17.777779,13.888889,0.0,0.70,1.33174,103.96608,1.33246,103.96649
39,Frankel Ave/Jalan Kembangan,650.0,10.000000,10.000000,13.333334,1.5,0.96,1.33174,103.96608,1.33246,103.96649
40,BEDOK NORTH AVE 3 NB AND BEDOK NORTH AVE 3 SB,337.0,12.777778,12.777778,14.166667,0.3,0.84,1.33174,103.96608,1.33246,103.96649
51,Upp Bukit Timah Rd/Dunearn Rd,394.0,10.833334,10.833334,13.888889,1.4,0.99,1.33174,103.96608,1.33246,103.96649
84,THOMSON RD SOUTHBOUND,316.0,11.944445,11.944445,10.833334,0.0,0.73,1.33174,103.96608,1.33246,103.96649
...,...,...,...,...,...,...,...,...,...,...,...
5446,Upper Bukit Timah Rd,96.0,16.666668,17.777779,16.666668,0.0,0.99,1.33174,103.96608,1.33246,103.96649
5460,KEPPEL RD WESTBOUND,567.0,16.388890,16.388890,15.555556,0.0,0.99,1.33174,103.96608,1.33246,103.96649
5469,ANG MO KIO AVE 3 EASTBOUND,46.0,8.333334,8.333334,11.111112,1.2,0.99,1.33174,103.96608,1.33246,103.96649
5500,Marina PL,20.0,11.111112,11.111112,9.166667,0.0,0.99,1.33174,103.96608,1.33246,103.96649


In [39]:
unique_list = []
duplicated = []

for item in df["description"].tolist():
    if item in unique_list:
        #print(item)
        duplicated.append(item)
    else:
        unique_list.append(item)

In [40]:
print("Duplicate records: ", len(duplicated), "\nUnique records:    ", len(unique_list))

Duplicate records:  3964 
Unique records:     1538


**There should only be 1538 rows after we handled the duplicate data**

### Aggregating the duplicate rows
**Sum the length feature, calculate the weighted average for speed, speedUncapped, freeFlow**

In [41]:
road_aggregated_data = {}

for _, row in duplicate_df.iterrows():
    road_name = row['description']
    start_lat = row['start_lat']
    start_lon = row['start_lon']
    end_lat = row['end_lat']
    end_lon = row['end_lon']
    length = row['length']
    speed = row['speed']
    speed_uncapped = row['speedUncapped']
    free_flow = row['freeFlow']
    jam_factor = row['jamFactor']
    confidence = row['confidence']
    
    if road_name in road_aggregated_data:
        (
            cumulative_length,
            cumulative_weighted_speed,
            cumulative_weighted_speed_uncapped,
            cumulative_weighted_free_flow,
            cumulative_weighted_jam_factor
        ) = road_aggregated_data[road_name][4:9]
        
        cumulative_length += length
        cumulative_weighted_speed += length * speed
        cumulative_weighted_speed_uncapped += length * speed_uncapped
        cumulative_weighted_free_flow += length * free_flow
        cumulative_weighted_jam_factor += length * jam_factor
        
        road_aggregated_data[road_name] = (
            start_lat,
            start_lon,
            end_lat,
            end_lon,
            cumulative_length,
            cumulative_weighted_speed,
            cumulative_weighted_speed_uncapped,
            cumulative_weighted_free_flow,
            cumulative_weighted_jam_factor,
            confidence
        )
    else:
        road_aggregated_data[road_name] = (
            start_lat,
            start_lon,
            end_lat,
            end_lon,
            length,
            length * speed,
            length * speed_uncapped,
            length * free_flow,
            length * jam_factor,
            confidence
        )

road_average_speeds = {}

for road_name, (
    start_lat,
    start_lon,
    end_lat,
    end_lon,
    cumulative_length,
    cumulative_weighted_speed,
    cumulative_weighted_speed_uncapped,
    cumulative_weighted_free_flow,
    cumulative_weighted_jam_factor,
    confidence
) in road_aggregated_data.items():
    average_speed = cumulative_weighted_speed / cumulative_length
    average_speed_uncapped = cumulative_weighted_speed_uncapped / cumulative_length
    average_free_flow = cumulative_weighted_free_flow / cumulative_length
    average_weighted_jam_factor = cumulative_weighted_jam_factor / cumulative_length
    
    road_average_speeds[road_name] = (
        start_lat,
        start_lon,
        end_lat,
        end_lon,
        cumulative_length,
        average_speed,
        average_speed_uncapped,
        average_free_flow,
        average_weighted_jam_factor,
        confidence
    )

In [42]:
result_df = pd.DataFrame(list(road_average_speeds.values()), columns=['start_lat', 'start_lon', 'end_lat', 'end_lon', 'length', 'speed', 'speedUncapped', 'freeFlow', 'jamFactor', 'confidence'])
result_df.insert(0, 'description', list(road_average_speeds.keys()))

In [43]:
result_df

,description,start_lat,start_lon,end_lat,end_lon,length,speed,speedUncapped,freeFlow,jamFactor,confidence
0,Changi South Ave 2,1.33174,103.96608,1.33246,103.96649,1631.0,9.147422,9.147422,9.661762,0.768670,0.70
1,Benoi Rd,1.33174,103.96608,1.33246,103.96649,11861.0,10.669187,10.669187,12.753281,0.869244,0.87
2,Jalan Kebun Limau,1.33174,103.96608,1.33246,103.96649,589.0,8.481890,8.481890,12.682041,2.158574,0.99
3,Jurong Town Hall Rd/Exit 13,1.33174,103.96608,1.33246,103.96649,2963.0,20.467152,20.467152,21.178986,0.580695,0.99
4,Syed Alwi Rd,1.33174,103.96608,1.33246,103.96649,176.0,7.058081,7.058081,8.222854,0.338068,0.90
...,...,...,...,...,...,...,...,...,...,...,...
1222,N Buona Vista Rd/Exit 8,1.33174,103.96608,1.33246,103.96649,3050.0,21.559746,21.559746,21.485065,0.207738,0.99
1223,Pioneer Circus/Pioneer Rd,1.33174,103.96608,1.33246,103.96649,3457.0,13.642047,13.642047,14.629737,0.500000,0.99
1224,Fort Rd/Exit 13,1.33174,103.96608,1.33246,103.96649,2250.0,22.150001,22.150001,23.538889,0.830400,0.99
1225,Penang Rd/Somerset Rd,1.33174,103.96608,1.33246,103.96649,172.0,5.394057,5.394057,6.143411,0.202326,0.90


**Verify the aggregated data**

In [44]:
result_df.loc[result_df['description']=='1/Jalan Tun Abdul Razak']

,description,start_lat,start_lon,end_lat,end_lon,length,speed,speedUncapped,freeFlow,jamFactor,confidence
950,1/Jalan Tun Abdul Razak,1.33174,103.96608,1.33246,103.96649,892.0,7.473842,7.473842,11.456153,2.725336,0.7


In [45]:
duplicate_df.loc[df["description"]=="1/Jalan Tun Abdul Razak"]

,description,length,speed,speedUncapped,freeFlow,jamFactor,confidence,start_lat,start_lon,end_lat,end_lon
2222,1/Jalan Tun Abdul Razak,758.0,7.777778,7.777778,12.222222,3.0,0.73,1.33174,103.96608,1.33246,103.96649
5193,1/Jalan Tun Abdul Razak,86.0,5.555556,5.555556,7.222222,1.1,0.72,1.33174,103.96608,1.33246,103.96649
5418,1/Jalan Tun Abdul Razak,48.0,6.111111,6.111111,6.944445,1.3,0.70,1.33174,103.96608,1.33246,103.96649


**Combine the unique records with the aggregated records**

In [46]:
clean_df = pd.concat([unique_df, result_df])

In [47]:
clean_df.head()

,description,length,speed,speedUncapped,freeFlow,jamFactor,confidence,start_lat,start_lon,end_lat,end_lon
1,ECP/East Coast Pkwy,44.0,17.777779,17.777779,13.888889,0.0,0.70,1.33174,103.96608,1.33246,103.96649
39,Frankel Ave/Jalan Kembangan,650.0,10.000000,10.000000,13.333334,1.5,0.96,1.33174,103.96608,1.33246,103.96649
40,BEDOK NORTH AVE 3 NB AND BEDOK NORTH AVE 3 SB,337.0,12.777778,12.777778,14.166667,0.3,0.84,1.33174,103.96608,1.33246,103.96649
51,Upp Bukit Timah Rd/Dunearn Rd,394.0,10.833334,10.833334,13.888889,1.4,0.99,1.33174,103.96608,1.33246,103.96649
84,THOMSON RD SOUTHBOUND,316.0,11.944445,11.944445,10.833334,0.0,0.73,1.33174,103.96608,1.33246,103.96649


In [48]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1538 entries, 1 to 1226
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   description    1538 non-null   object 
 1   length         1538 non-null   float64
 2   speed          1538 non-null   float64
 3   speedUncapped  1538 non-null   float64
 4   freeFlow       1538 non-null   float64
 5   jamFactor      1538 non-null   float64
 6   confidence     1538 non-null   float64
 7   start_lat      1538 non-null   float64
 8   start_lon      1538 non-null   float64
 9   end_lat        1538 non-null   float64
 10  end_lon        1538 non-null   float64
dtypes: float64(10), object(1)
memory usage: 144.2+ KB


**length (and jamFactor and confidence too perhaps), remove 12 and**

**use the shape for unique identification of road name, check if it is true**

In [49]:
clean_df = clean_df.drop("confidence", axis=1)

In [59]:
sorted_df = clean_df.sort_values("description")

In [63]:
sorted_df['road_id'] = pd.factorize(sorted_df['description'])[0] + 1

In [55]:
clean_df.corr()

,length,speed,speedUncapped,freeFlow,jamFactor,start_lat,start_lon,end_lat,end_lon,road_id
length,1.000000,0.174647,0.164839,0.215871,-0.013400,NaN,NaN,NaN,NaN,0.018824
speed,0.174647,1.000000,0.991400,0.910031,-0.608016,NaN,NaN,NaN,NaN,0.058676
speedUncapped,0.164839,0.991400,1.000000,0.893933,-0.615057,NaN,NaN,NaN,NaN,0.047881
freeFlow,0.215871,0.910031,0.893933,1.000000,-0.295453,NaN,NaN,NaN,NaN,0.092960
jamFactor,-0.013400,-0.608016,-0.615057,-0.295453,1.000000,NaN,NaN,NaN,NaN,0.047123
start_lat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
start_lon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
end_lat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
end_lon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
road_id,0.018824,0.058676,0.047881,0.092960,0.047123,NaN,NaN,NaN,NaN,1.000000


In [64]:
sorted_df

,description,length,speed,speedUncapped,freeFlow,jamFactor,start_lat,start_lon,end_lat,end_lon,road_id
950,1/Jalan Tun Abdul Razak,892.0,7.473842,7.473842,11.456153,2.725336,1.33174,103.96608,1.33246,103.96649,1
1100,12,52.0,13.888889,15.833334,13.888889,0.000000,1.33174,103.96608,1.33246,103.96649,2
4915,15A/Woodsville Flyover,75.0,5.000000,5.000000,7.777778,2.000000,1.33174,103.96608,1.33246,103.96649,3
609,17/Jalan Pekeliling,931.0,13.784461,13.784461,15.210945,0.940602,1.33174,103.96608,1.33246,103.96649,4
1189,188/Jalan Lingkaran Dalam Johor Bahru/Johor Ba...,747.0,9.095270,9.095270,14.194557,3.003614,1.33174,103.96608,1.33246,103.96649,5
...,...,...,...,...,...,...,...,...,...,...,...
1035,Yishun Ring Rd (South),1516.0,9.083847,9.083847,12.122728,1.767084,1.33174,103.96608,1.33246,103.96649,1534
204,Yuan Ching Rd,1157.0,9.324162,9.324162,11.461635,0.917113,1.33174,103.96608,1.33246,103.96649,1535
1096,Yuan Ching Rd/Exit 15B,1421.0,19.782432,19.782432,21.042695,0.979803,1.33174,103.96608,1.33246,103.96649,1536
1119,Yuan Ching Rd/Jurong West St 51,2088.0,9.935079,9.935079,14.534643,2.623372,1.33174,103.96608,1.33246,103.96649,1537


In [65]:
metadata = sorted_df[["road_id", "description", "length", "start_lat", "start_lon", "end_lat", "end_lon"]]

In [66]:
metadata

,road_id,description,length,start_lat,start_lon,end_lat,end_lon
950,1,1/Jalan Tun Abdul Razak,892.0,1.33174,103.96608,1.33246,103.96649
1100,2,12,52.0,1.33174,103.96608,1.33246,103.96649
4915,3,15A/Woodsville Flyover,75.0,1.33174,103.96608,1.33246,103.96649
609,4,17/Jalan Pekeliling,931.0,1.33174,103.96608,1.33246,103.96649
1189,5,188/Jalan Lingkaran Dalam Johor Bahru/Johor Ba...,747.0,1.33174,103.96608,1.33246,103.96649
...,...,...,...,...,...,...,...
1035,1534,Yishun Ring Rd (South),1516.0,1.33174,103.96608,1.33246,103.96649
204,1535,Yuan Ching Rd,1157.0,1.33174,103.96608,1.33246,103.96649
1096,1536,Yuan Ching Rd/Exit 15B,1421.0,1.33174,103.96608,1.33246,103.96649
1119,1537,Yuan Ching Rd/Jurong West St 51,2088.0,1.33174,103.96608,1.33246,103.96649


In [67]:
metadata.to_csv("metadata.csv", index=False)